<a href="https://colab.research.google.com/github/sanu00007/Deep_Learning/blob/main/CNN_PRACTICE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#Convert MNIST Image Files into tensor of 4-Dimensions (# of images ,Height,Width,color channel)
transforms = transforms.ToTensor()
train_data = datasets.MNIST(root='/data',train=True,download=True,transform=transforms)
test_data = datasets.MNIST(root='/data',train=False,download=True,transform=transforms)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 157058981.56it/s]


Extracting /data/MNIST/raw/train-images-idx3-ubyte.gz to /data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 47616231.85it/s]

Extracting /data/MNIST/raw/train-labels-idx1-ubyte.gz to /data/MNIST/raw



...

100%|██████████| 1648877/1648877 [00:00<00:00, 43892028.72it/s]

Extracting /data/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 7605001.50it/s]

Extracting /data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/MNIST/raw



In [4]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [5]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
#create a small batch size for images.....lets say 10
train_loader = DataLoader(train_data,batch_size=10,shuffle = True)
test_loader = DataLoader(test_data,batch_size=10,shuffle = False)

In [12]:
#define our cnn model
#describe convolution layer ans there are 2 layer

#just an example

conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=3,stride=1)
conv2 = nn.Conv2d(in_channels=6,out_channels=16,kernel_size=3,stride=1) #6 output from conv1 are input of conv2

In [13]:
#Grab 1 MNIST record/image
for i,(X_train,y_train) in enumerate(train_data):
    break

In [24]:
X_train.shape

torch.Size([1, 28, 28])

In [30]:
x = X_train.view(1,1,28,28) #4Dimension

In [31]:
#Perform out first convolution
x = F.relu(conv1(x)) #Rectify Linear unit for our activation fuction
# 1 single image, 6 is filter we asked for, 26x26
x.shape


torch.Size([1, 6, 26, 26])

In [32]:
#pooling layer
x = F.max_pool2d(x,2,2) #max_pool
x.shape

#since pooling is 2x2 it change to 13x13 from 26x26

torch.Size([1, 6, 13, 13])

In [33]:
#Do our second convolution ayer
x = F.relu(conv2(x))
x.shape

torch.Size([1, 16, 11, 11])

In [34]:
x = F.max_pool2d(x,2,2)
x.shape

torch.Size([1, 16, 5, 5])

In [35]:
((28-2)/2-2)/2

5.5

In [64]:
#Model Class
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=3,stride=1)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=16,kernel_size=3,stride=1)
        #Fully Conected layer
        self.fc1 = nn.Linear(in_features=16*5*5,out_features=120)
        self.fc2 = nn.Linear(in_features=120,out_features=84)
        self.fc3 = nn.Linear(in_features=84,out_features=10)

    def forward(self,X):
      #first pass
      X = F.relu(self.conv1(X))
      X = F.max_pool2d(X,2,2)
      #second pass
      X = F.relu(self.conv2(X))
      X = F.max_pool2d(X,2,2)

      X = X.view(-1,16*5*5)
      #Fully connected layer
      X = F.relu(self.fc1(X))
      X = F.relu(self.fc2(X))
      X = self.fc3(X)
      return F.log_softmax(X,dim=1)

In [65]:
#Create an instance of our model
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [66]:
#Loss Function optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001) #smaller the learning rate, longer its gonna take to train



In [67]:
import time
start_time = time.time()

#Create variables to track things
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

#Train our model
for i in range(epochs):
  trn_corr = 0
  tst_corr = 0

  #Train
  for b,(X_train,y_train) in enumerate(train_loader):
    b+=1 #start out batches ar 1
    #Apply model
    y_pred = model(X_train) #Not flattended
    loss = criterion(y_pred,y_train)
    predicted = torch.max(y_pred.data,1)[1] #add up the number of current predictions
    batch_corr = (predicted == y_train).sum() # how many we got correct from specfic batch
    trn_corr += batch_corr

    #Backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #Print out some results
    if b%600 == 0: #so each batch is 600
      print(f'epoch:{i} bath: {b}  loss:{loss.item()}')

  train_losses.append(loss)
  train_correct.append(trn_corr)


#Test
with torch.no_grad(): #NO gradient so we dont update
  for b,(X_test,y_test) in enumerate(test_loader):
    y_val = model(X_test)
    predicted = torch.max(y_val.data,1)[1] # adding up correct prediction
    tst_corr += (predicted == y_test).sum()

  loss = criterion(y_val,y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)



current_time = time.time()
print(f'Training took:{(current_time-start_time)/60} minutes')


epoch:0 bath: 600  loss:0.1623610556125641
epoch:0 bath: 1200  loss:0.1502392590045929
epoch:0 bath: 1800  loss:0.4744560718536377
epoch:0 bath: 2400  loss:0.14238706231117249
epoch:0 bath: 3000  loss:0.007758188061416149
epoch:0 bath: 3600  loss:0.3836284875869751
epoch:0 bath: 4200  loss:0.0038223876617848873
epoch:0 bath: 4800  loss:0.0021286322735249996
epoch:0 bath: 5400  loss:0.0569545142352581
epoch:0 bath: 6000  loss:0.00038789428072050214
epoch:1 bath: 600  loss:0.06289136409759521
epoch:1 bath: 1200  loss:0.010614877566695213
epoch:1 bath: 1800  loss:0.03243611007928848
epoch:1 bath: 2400  loss:0.012448625639081001
epoch:1 bath: 3000  loss:0.000640809943433851
epoch:1 bath: 3600  loss:0.0020938280504196882
epoch:1 bath: 4200  loss:0.3140248656272888
epoch:1 bath: 4800  loss:0.020231451839208603
epoch:1 bath: 5400  loss:0.0031914091669023037
epoch:1 bath: 6000  loss:0.0009488927898928523
epoch:2 bath: 600  loss:0.04242878407239914
epoch:2 bath: 1200  loss:0.000786997377872467
